## Running **spike sorting** pipelines:

A spike sorting pipeline for neuropixels should consist of:

 - preprocessing (phase_shifting, filtering, denoising..)
 - motion correction
 - the actual sorting (assigning spikes to units)
 - waveform extraction
 - metric computation

and some other optional steps like reading __sycnchronization channels__, __manual curation__ and result inspection.

### ecephys spike sorting

Check the usage instructions [**here**](https://github.com/jenniferColonell/ecephys_spike_sorting#usage).

__ecephys__ is installed on the _course computers_; the installation instructions are [**here**](https://github.com/jenniferColonell/ecephys_spike_sorting#installation-with-anaconda-and-kilosort4); don't forget to install _CatGT_ and set the paths in the ``ecephys_spike_sorting\ecephys_spike_sorting\scripts\create_input_json.py`` file if you are installing from scratch. 

---

To **configure** the pipeline to run the test data you need to edit the file ``sglx_multi_run_pipeline.py`` in the ``ecephys_spike_sorting\ecephys_spike_sorting\scripts`` directory.

In the _course computers_ that file is in: ``C\Users\np_user\Desktop\software_downloads\ecephys_course\ecephys_spike_sorting\ecephys_spike_sorting\scripts`` 

The file (**sglx_multi_run_pipeline.py**) controls most aspects of the pipeline. Some things to check:
 1. __line 67__: ``npx_directory = r'<PARENT DIRECTORY OF THE INPUT (e.g. C:\SGL_DATA)' `` 
 2. __line 95 to 97__: ``run_specs = [['<FILEPART NAME BEFORE THE GATE WITHOUT UNDERSCORE`, '<GATE NUMBER', '0,0', '0', '0', ['cortex','thalamus','thalamus'] ]``  specifies the file and gate to run, add multiple lines to concatenate gates or sessions

 3. __line 105__: ``catGT_dest = r'<PATH TO AN EMPTY FOLDER>'`` don't forget to create the folder. this is the output path.
 4. __line 216__: ``json_directory = r'<PATH TO THE OUTPUT WHERE TO STORE JSON FILES>'`` this can be the same folder as point 3 or another if you want the files to be separate.
 5. __line 135 to 136__: ``obx_present = True`` set to true to analyze the OneBox channels ``ni_present = False`` to analyze obx channels
 6. __line 137__: ``ni_obx_extract_string = '-xa=1,0,1,1,3,50`` extract the 50 ms pulses that mark trial start    

---

To **run**:
 - open a terminal where you can run conda commands. 
 - activate the environment ``conda activate ecephys_ks4`` _in the course_.
 - go to the folder that has the ``sglx_multi_run_pipeline.py`` file using ``cd <FOLDERPATH>``
 - run the pipeline ``python sglx_multi_run_pipeline.py``


 ---

 To **visualize** the results:

You can visualize with [**phy**](https://github.com/cortex-lab/phy).

 - open a terminal where you can run conda commands. 
 - activate the environment ``conda activate phy2`` _in the course_.
 - go to the output folder using ``cd <FOLDERPATH>`` you want the folder that has the **params.py** file.
 - open phy using the command **``phy template-gui params.py``**


You can analyze the visual responses in the notebook 03_visual_responses.ipynb







### Combining tools to build a pipeline

## Building a CatGT command line for preprocessing
Preprocessing can also be done in Spikeinterface -- here we use CatGT to handle a few SGLX details more easily.
Here we build a CatGT command line to process your data, then look at the result with the SpikeGLX viewer. 
For more details on CatGT, see the help [**here**](https://billkarsh.github.io/SpikeGLX/CatGT_help/CatGT_ReadMe.html).

This is also a general model for incorporating command line or other compiled tools into a simple python notebook or script.

In [10]:
import os
import subprocess
from pathlib import Path
os_str='Windows'

# set the parent path for the software on your computer
software_download_path = r'C:\Users\labadmin\Desktop\software'

# parent folder of the data
data_parent = r'C:\SGL_DATA\Group4_bank0_kc\KS001_V1_011526_KC_g0'

# run name for the acquired data (don't include any gate info)
run_name = 'KS001_V1_011526_KC_g0'

# gate index (for myrun_g0_t0.imec0.ap.bin, the gate = 0)
gate = 0

# destination parent
dest_parent = r'C:\Users\np_user\Desktop\test_data\output'
Path(dest_parent).mkdir(parents=True,exist_ok=True)


    
# time to analyze and sort 
maxsecs = 600

# extraction parameters for stimulus times. parameters: stream type(js), stream index(ip), channel in file, threshold1, threshold2, duration
obx_ex_str = '-xa=1,0,1,1,3,50'

dest_dir = os.path.join(dest_parent,f'{run_name}_g{gate}')
if not os.path.isdir(dest_dir):
    os.mkdir(dest_dir)

cmd_parts = list()
cmd_parts.append(os.path.join(software_download_path,'ecephys_course/CatGT-win/CatGT.exe'))
cmd_parts.append(f'-dir={data_parent} -run={run_name} -g={gate} -t=0,0 -prb_fld')
cmd_parts.append(f'-prb=0 -ap')
cmd_parts.append(f'-apfilter=butter,12,300,10000 -gblcar -gfix=0.40,0.10,0.02')
#cmd_parts.append(f'-ob {obx_ex_str}')
cmd_parts.append(f'-maxsecs={maxsecs}')
cmd_parts.append(f'-out_prb_fld -no_catgt_fld -dest={dest_dir}')
                        

catGT_cmd = ' '        # use space as the separator for the command parts
catGT_cmd = catGT_cmd.join(cmd_parts[1:len(cmd_parts)]) # these are the parameters
if os_str=='linux':
    # enclose the params in single quotes, so curly braces will not be interpreted by Linux
    catGT_cmd = f"{cmd_parts[0]} '{catGT_cmd}'"
else:
    catGT_cmd = f"{cmd_parts[0]} {catGT_cmd}"

print(catGT_cmd)
subprocess.Popen(catGT_cmd,shell='False').wait()
print('CatGT complete, check CatGT.log in notebook folder for errors.')



C:\Users\labadmin\Desktop\software\ecephys_course/CatGT-win/CatGT.exe -dir=C:\SGL_DATA\Group4_bank0_kc\KS001_V1_011526_KC_g0 -run=KS001_V1_011526_KC_g0 -g=0 -t=0,0 -prb_fld -prb=0 -ap -apfilter=butter,12,300,10000 -gblcar -gfix=0.40,0.10,0.02 -maxsecs=600 -out_prb_fld -no_catgt_fld -dest=C:\Users\np_user\Desktop\test_data\output\KS001_V1_011526_KC_g0_g0
CatGT complete, check CatGT.log in notebook folder for errors.


### Running Spike interface on preprocessed data

The processed data will be saved to: ``<destination parent>/<run_name>_g<gate>``. Open it up in the SpikeGLX viewer and look for any problems. Is the background subtraction sufficient? Are any major artifacts removed?

After checking that the preprocessed data looks OK, move to sorting a single shank, using spike interface


Check the instructions [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/overview.html).

__spikeinterface__ is installed in the npix-analysis environment on the _course computers_; the installation instructions are [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/installation.html).

Be sure to checkout the other [tutorials](https://spikeinterface.readthedocs.io/en/0.93.0/getting_started/plot_getting_started.html#).

---

We will build and run a pipeline on this notebook, it is based on the [how to guide](https://spikeinterface.readthedocs.io/en/latest/how_to/analyze_neuropixels.html). 


In [9]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# parameters -- these only pertain to the sorter, because the data is already preprocessed.
# in this example, we keep the filtering and whitening in Kilosort 4. This is to keep the native Ks4 whitening (to avoid scaling problems)

ks4_params = si.get_default_sorter_params('kilosort4')
ks4_params['do_CAR'] = False # skip CAR in kilosort
job_kwargs = dict(n_jobs=-1, chunk_duration='1s', progress_bar=True) # how to chunk and process data

sort_folder = os.path.join(dest_dir,f'ks4_out')  # this folder is created by KS4; existing folder cannot be overwritten

# load spikeglx data, pointing to the destination folder created above
stream_names, stream_ids = si.get_neo_streams('spikeglx', dest_dir)

print(f'There are streams {stream_names} in folder {dest_dir}')


FileNotFoundError: No appropriate combination of .meta and .bin files were detected in C:\Users\np_user\Desktop\test_data\output\KS001_V1_011526_KC_g0_g0

In [ ]:
# SET the stream parameter for the file you want to sort here

selected_stream = 'imec0.ap'

In [ ]:
print(f'All Kilosort4 parameters are: ')
for k in ks4_params.keys():
    print(f'   - {k}: {ks4_params[k]}')

In [ ]:
# sort the stream for the selected shank

rec = si.read_spikeglx(dest_dir, stream_name=selected_stream, load_sync_channel=False)

# run ks4
sorting = si.run_sorter('kilosort4', rec, folder=sort_folder,
                        docker_image=False, verbose=True, **ks4_params)
sorting = si.read_sorter_folder(sort_folder)

# phy output from ks4 is saved to sort_folder/sorter_output

In [ ]:
# calculate average waveforms and metrics.

sorting = si.read_sorter_folder(sort_folder)

analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, format="memory")
# compute waveforms 
analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
analyzer.compute("waveforms",  ms_before=1.5, ms_after=1.5, **job_kwargs)
analyzer.compute("templates", operators=["average", "median", "std"])
analyzer.compute("noise_levels")
analyzer.compute("correlograms")
analyzer.compute("unit_locations")
analyzer.compute("spike_amplitudes", **job_kwargs)

# quality metrics
metric_names=['firing_rate', 'presence_ratio', 'snr', 'isi_violation', 'amplitude_cutoff']
metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)

# all of the analyzer computed quantitites are saved in sort_folder/analyzer/extensions
analyzer_saved = analyzer.save_as(folder=os.path.join(sort_folder, "analyzer"), format="binary_folder")
